In [11]:
!apt-get install iverilog -y
!iverilog -v

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
iverilog is already the newest version (11.0-1.1).
0 upgraded, 0 newly installed, 0 to remove and 2 not upgraded.
Icarus Verilog version 11.0 (stable) ()

Copyright 1998-2020 Stephen Williams

  This program is free software; you can redistribute it and/or modify
  it under the terms of the GNU General Public License as published by
  the Free Software Foundation; either version 2 of the License, or
  (at your option) any later version.

  This program is distributed in the hope that it will be useful,
  but WITHOUT ANY WARRANTY; without even the implied warranty of
  MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
  GNU General Public License for more details.

  You should have received a copy of the GNU General Public License along
  with this program; if not, write to the Free Software Foundation, Inc.,
  51 Franklin Street, Fifth Floor, Boston, MA 02110-1301, USA.

iveril

In [12]:
import os
import subprocess

with open("sequence_detector_tb.v", "w") as f:
    f.write("""
`timescale 1ns / 1ps
module sequence_detector_tb;
    reg clk, rst_n, in;
    wire out;
    sequence_detector uut (.clk(clk), .rst_n(rst_n), .in(in), .out(out));

    initial begin
        clk = 0;
        forever #5 clk = ~clk;
    end

    integer errors = 0;
    task check(input expected);
        begin
            #1; //
            if (out !== expected) begin
                $display("Time %t | Error: Input=%b, Expected=%b, Got=%b", $time, in, expected, out);
                errors = errors + 1;
            end
        end
    endtask

    initial begin
        rst_n = 0; in = 0;
        #15 rst_n = 1;

        // sequence detect
        @(negedge clk) in = 1; check(0); // S_IDLE -> S_1
        @(negedge clk) in = 0; check(0); // S_1 -> S_10
        @(negedge clk) in = 1; check(0); // S_10 -> S_101
        @(negedge clk) in = 1; check(1); // S_101 -> S_1 (检测成功!)

        // testing overlapping
        @(negedge clk) in = 0; check(0); // S_1 -> S_10
        @(negedge clk) in = 1; check(0); // S_10 -> S_101
        @(negedge clk) in = 1; check(1); // S_101 -> S_1 (检测成功!)

        @(negedge clk) in = 0; check(0);
        #10;

        if (errors == 0) $display("Mismatches: 0");
        else $display("Mismatches: %d", errors);
        $finish;
    end
endmodule
""")

code = """
`timescale 1ns / 1ps
module sequence_detector (
    input wire clk,
    input wire rst_n,
    input wire in,
    output reg out
);
    localparam [1:0] S_IDLE = 2'd0,
                     S_1    = 2'd1,
                     S_10   = 2'd2,
                     S_101  = 2'd3;

    reg [1:0] state, next_state;

    // update statues
    always @(posedge clk or negedge rst_n)
        if (!rst_n) state <= S_IDLE;
        else state <= next_state;

    // next statues and output logic
    always @(*) begin
        next_state = S_IDLE;
        out = 0; // defualt output
        case (state)
            S_IDLE: next_state = (in) ? S_1 : S_IDLE;
            S_1:    next_state = (in) ? S_1 : S_10;
            S_10:   next_state = (in) ? S_101 : S_IDLE;
            S_101:  begin
                if (in) begin
                    out = 1;
                    next_state = S_1;
                end else begin
                    next_state = S_10; // ...1010 -> 10
                end
            end
        endcase
    end
endmodule
"""
with open("sequence_detector_manual.v", "w") as f:
    f.write(code)

output_dir = "./manual_verification"
if not os.path.exists(output_dir): os.makedirs(output_dir)

print("Compiling and Simulating...")
cmd = "iverilog -o manual_test sequence_detector_manual.v sequence_detector_tb.v && vvp manual_test"
process = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

print(process.stdout)
if "Mismatches: 0" in process.stdout:
    print("\n Success, Manual design verified.")
else:
    print("\n Failed, Check output above.")

Compiling and Simulating...
Mismatches: 0


 Success, Manual design verified.
